<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# Python for Algorithmic Trading

**Chapter 04 &mdash; Vectorized Backtesting**

## Strategies based on Simple Moving Averages

### Getting into the Basics 

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# data from http://hilpisch.com/pyalgo_eikon_eod_data.csv
raw = pd.read_csv('pyalgo_eikon_eod_data.csv', index_col=0, parse_dates=True).dropna()

In [ ]:
raw.info()

In [ ]:
data = pd.DataFrame(raw['EUR='])
data.rename(columns={'EUR=': 'price'}, inplace=True)
data['SMA1'] = data['price'].rolling(42).mean()
data['SMA2'] = data['price'].rolling(252).mean()


In [ ]:
data.tail()

In [ ]:
# sets up the matplotlib to work interactively
%matplotlib inline
from pylab import mpl, plt
plt.style.use('seaborn')
mpl.rcParams['font.family'] = 'serif'

In [ ]:
data.plot(title='AAPL stock price | 42 & 252 days SMAs', figsize=(10, 6));
# plt.savefig('../../images/ch04/sma_plot_1.png')

In [ ]:
data['position'] = np.where(data['SMA1'] > data['SMA2'], 1, -1)

In [ ]:
data.dropna(inplace=True)

In [ ]:
data['position'].plot(ylim=[-1.1, 1.1], title='Market Positioning', figsize=(10, 6));
# plt.savefig('../../images/ch04/sma_plot_2.png')

In [ ]:
# get "Log returns". 
# calculation that assumes a continuously compounding rate of return.
data['returns'] = np.log(data['price'] / data['price'].shift(1))

In [ ]:
data['returns'].hist(bins=35, figsize=(10, 6));
# plt.savefig('../../images/ch04/sma_plot_3.png')

In [ ]:
data['strategy'] = data['position'].shift(1) * data['returns']

In [ ]:
data[['returns', 'strategy']].sum()

In [ ]:
data.tail()

In [ ]:
start_price = data['price'].iloc[0]
end_price = data['price'].iloc[-1]

print(start_price, end_price)

def calc_gross_rate(start, end):
    return (end - start) / start

rate = calc_gross_rate(start_price, end_price)
percent = rate * 100

print(f"rate {rate} (percent: {percent}%)")

In [ ]:
data[['returns', 'strategy']].sum()


In [ ]:
data[['returns', 'strategy']].sum().apply(np.exp)

In [ ]:
data[['returns', 'strategy']].dropna().cumsum().apply(np.exp).plot(figsize=(10, 6));
# plt.savefig('../../images/ch04/sma_plot_4.png')

#### Calc the annualized mean return

In [ ]:
data[['returns', 'strategy']].mean() * 252

In [ ]:
np.exp(data[['returns', 'strategy']].mean() * 252) - 1

### Calc the annualized standard deviation

In [ ]:
data[['returns', 'strategy']].std() * 252 ** 0.5

In [ ]:
np.exp(data[['returns', 'strategy']].std() * 252 ** 0.5) - 1

### Calc the max and the longest drawdown

1. calc the cumret and cummax (then plot)
2. the max drawdown is the greatest diff between the two
3. calc the longest drawdown

In [ ]:
print(data['strategy'].cumsum())
data['cumulative_return'] = data['strategy'].cumsum().apply(np.exp)

In [ ]:
data['cumulative_max'] = data['cumulative_return'].cummax()

In [ ]:
data[['cumulative_return', 'cumulative_max']].dropna().plot(figsize=(10, 6));

# plt.savefig('../../images/ch04/sma_plot_5.png')

In [ ]:
drawdown = data['cumulative_max'] - data['cumulative_return']

In [ ]:
drawdown.max()

In [ ]:
temp = drawdown[drawdown == 0]
temp

In [ ]:
periods = (temp.index[1:].to_pydatetime() -
           temp.index[:-1].to_pydatetime())

In [ ]:
len(periods)

In [ ]:
periods[12:15]

In [ ]:
periods.max()

### Generalizing the Approach

In [ ]:
import SMAVectorBackTester as SMA

In [ ]:
smabt = SMA.SMAVectorBackTester('EUR=', 42, 252, '2010-1-1', '2019-12-31') 

In [ ]:
smabt.run_strategy()

In [ ]:
smabt.optimize_parameters((30, 50, 2),  (200, 300, 2))

In [ ]:
smabt.plot_results()
# plt.savefig('../../images/ch04/sma_plot_6.png')

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:training@tpq.io">training@tpq.io</a>